In [22]:
!nvidia-smi

Sat Dec  8 18:57:49 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.130                Driver Version: 384.130                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:03:00.0  On |                  N/A |
|  0%   45C    P8    17W / 240W |   8103MiB /  8108MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 1080    Off  | 00000000:82:00.0 Off |                  N/A |
|  0%   33C    P8    16W / 210W |      2MiB /  8114MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [ ]:
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [ ]:
import numpy as np
import pandas as pd

In [23]:
import skimage.io
from skimage.transform import resize
from imgaug import augmenters as iaa
from tqdm import tqdm
import PIL
from PIL import Image
import cv2
from sklearn.utils import class_weight, shuffle

import warnings
warnings.filterwarnings("ignore")


#for NN
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
import keras
from keras.models import Model
import tensorflow as tf 


from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split



# 1. Load Data

In [31]:
# Load dataset info
path_to_train = '../ceph-data/kaggle-protein/train/'
train_labels = pd.read_csv('..ceph-data/kaggle-protein/train.csv')

FileNotFoundError: File b'..ceph-data/kaggle-protein/train.csv' does not exist

In [30]:
train_dataset_info = []
for name, labels in zip(train_labels['Id'], train_labels['Target'].str.split(' ')):
    train_dataset_info.append({
        'path': os.path.join(path_to_train, name),
        'labels': np.array([int(label) for label in labels])})
train_dataset_info = np.array(train_dataset_info)

NameError: name 'train_labels' is not defined

In [15]:
import pickle


with open('le.pkl', 'rb') as f:
    dict_label = pickle.load(f)

# 2. Model 

In [32]:
def create_model(input_shape, n_out):
    input_tensor = Input(shape=input_shape)
    
    
    base_model = InceptionV3(include_top=False,
                   weights='imagenet',
                   input_shape=input_shape)

#     base_model = ResNet50(include_top=False,
#                    weights='imagenet',
#                    input_shape=input_shape)

    
    
    batch_norm = BatchNormalization()(input_tensor)
    x = base_model(batch_norm)
    x = Conv2D(32, kernel_size=(1,1), activation='relu')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(n_out, activation='sigmoid')(x)
    model = Model(input_tensor, output)
    
    return model

### Tensorboard

In [24]:
from clr_callback import CyclicLR

In [26]:
SIZE = 299
N_CLASS = 28 
HOLD_OUT_part = 0.15
epochs = 10

proba_threshold=0.2

In [33]:
model_name = 'ResNet'
batch_size = 16

checkpoint = ModelCheckpoint('./models/InceptionV3.{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                                   verbose=1, mode='auto', epsilon=0.0001)
#clr = CyclicLR(base_lr=0.000005, max_lr=0.0005, step_size=2000)
#tensorboard = make_tensorboard(batch_size, model_name)

early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=6)
#callbacks_list = [checkpoint, clr, , tensorboard]

callbacks_list = [checkpoint, early, reduceLROnPlat]

In [29]:
# split data into train, valid
indexes = np.arange(train_dataset_info.shape[0])
np.random.shuffle(indexes)
train_indexes, valid_indexes = train_test_split(indexes, test_size=HOLD_OUT_part, random_state=8)

# create train and valid datagens
train_generator = DataGenerator.create_train(
    train_dataset_info[train_indexes], batch_size, (SIZE,SIZE,3), augument=True)
validation_generator = DataGenerator.create_train(
    train_dataset_info[valid_indexes], 32, (SIZE,SIZE,3), augument=False)

NameError: name 'train_dataset_info' is not defined

In [19]:
model = create_model(
    input_shape=(SIZE,SIZE,3), 
    n_out=N_CLASS)

for layer in model.layers:
    layer.trainable = False
    
#last six layers will be trained
model.layers[-1].trainable = True
model.layers[-2].trainable = True
model.layers[-3].trainable = True
model.layers[-4].trainable = True
model.layers[-5].trainable = True
model.layers[-6].trainable = True

87916544/87910968 [==============================] - 85s 1us/step


ResourceExhaustedError: OOM when allocating tensor with shape[3,3,448,384]
	 [[Node: conv2d_91/kernel/Assign = Assign[T=DT_FLOAT, _class=["loc:@conv2d_91/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_91/kernel, conv2d_91/random_uniform)]]

Caused by op 'conv2d_91/kernel/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177

In [ ]:
model.compile(
    loss='binary_crossentropy', 
    optimizer=Adam(1e-03),
    metrics=['acc'])
#model.summary()

In [20]:
model.fit_generator(
    train_generator,
    steps_per_epoch=np.ceil(float(len(train_indexes)) / float(batch_size)),
    validation_data=validation_generator,
    validation_steps=np.ceil(float(len(valid_indexes)) / float(batch_size)),
    epochs=2, 
    verbose=1)

NameError: name 'model' is not defined

In [21]:
1

1